In [2]:
# Here is the Model for Anime recommendation:
# it will cover : Populationbased Recommendation

#Collaborative and content based Recommendation will be covered later on

In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("anime-dataset-2023.csv")


In [5]:
df.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [6]:
df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [8]:
popularity_df=df[["English name","Score","Scored By"]]

In [9]:
popularity_df

,English name,Score,Scored By
0,Cowboy Bebop,8.75,914193.0
1,Cowboy Bebop: The Movie,8.38,206248.0
2,Trigun,8.22,356739.0
3,Witch Hunter Robin,7.25,42829.0
4,Beet the Vandel Buster,6.94,6413.0
...,...,...,...
24900,UNKNOWN,UNKNOWN,UNKNOWN
24901,Blader Soul,UNKNOWN,UNKNOWN
24902,The First Order,UNKNOWN,UNKNOWN
24903,UNKNOWN,UNKNOWN,UNKNOWN


In [10]:
# Preprocessing

In [11]:
df=popularity_df

In [12]:
df.shape

(24905, 3)

In [13]:
df.isnull().sum()

English name    0
Score           0
Scored By       0
dtype: int64

In [29]:
df[df["English name"]=="Promise"]

,English name,Score,Scored By
10473,Promise,4.54,226.0
16636,Promise,6.07,120.0
17556,Promise,6.94,1083.0
24488,Promise,UNKNOWN,UNKNOWN
24824,Promise,UNKNOWN,UNKNOWN


In [31]:
df = df[~df.isin(['UNKNOWN']).any(axis=1)]


In [37]:
df[df["English name"]=="Promise"]

,English name,Score,Scored By
10473,Promise,4.54,226.0
16636,Promise,6.07,120.0
17556,Promise,6.94,1083.0


In [38]:
df.drop_duplicates()

,English name,Score,Scored By
0,Cowboy Bebop,8.75,914193.0
1,Cowboy Bebop: The Movie,8.38,206248.0
2,Trigun,8.22,356739.0
3,Witch Hunter Robin,7.25,42829.0
4,Beet the Vandel Buster,6.94,6413.0
...,...,...,...
24579,Mashle: Magic and Muscles - Mash Burnedead and...,5.84,1971.0
24590,Lee's Detective Agency: A Day in Lungmen - Hom...,5.99,129.0
24635,Slash,6.45,152.0
24831,One Piece: Recapping Fierce Fights! The Counte...,6.29,320.0


In [40]:
df["Scored By"] = pd.to_numeric(df["Scored By"], errors="coerce")

df=df[df["Scored By"]>=250]

/tmp/ipykernel_9133/3661060166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Scored By"] = pd.to_numeric(df["Scored By"], errors="coerce")


In [41]:
df

,English name,Score,Scored By
0,Cowboy Bebop,8.75,914193.0
1,Cowboy Bebop: The Movie,8.38,206248.0
2,Trigun,8.22,356739.0
3,Witch Hunter Robin,7.25,42829.0
4,Beet the Vandel Buster,6.94,6413.0
...,...,...,...
24528,Setting Sun,6.80,291.0
24539,One Piece: Recapping Fierce Fights! Zoro vs. a...,6.47,722.0
24557,Touhou Fantasy Kaleidoscope: The Memories of P...,7.78,662.0
24579,Mashle: Magic and Muscles - Mash Burnedead and...,5.84,1971.0


In [42]:
total_csv=pd.read_csv("anime-dataset-2023.csv")

In [43]:
total_csv.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [44]:
df.columns

Index(['English name', 'Score', 'Scored By'], dtype='object')

In [48]:
more_features=total_csv[["English name","Other name","Genres","Synopsis","Image URL"]]

In [47]:
#here we have extracted other features that could be used to show info related
#to the anime like genre to show, image,other name

In [50]:
more_features

,English name,Other name,Genres,Synopsis,Image URL
0,Cowboy Bebop,カウボーイビバップ,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,"Action, Sci-Fi","Another day, another bounty—such is the life o...",https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,トライガン,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,https://cdn.myanimelist.net/images/anime/10/19...
4,Beet the Vandel Buster,冒険王ビィト,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...
24900,UNKNOWN,无脑魔女,"Comedy, Fantasy, Slice of Life",No description available for this anime.,https://cdn.myanimelist.net/images/anime/1386/...
24901,Blader Soul,捕星司·源起,"Action, Adventure, Fantasy",No description available for this anime.,https://cdn.myanimelist.net/images/anime/1383/...
24902,The First Order,第一序列,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,https://cdn.myanimelist.net/images/anime/1130/...
24903,UNKNOWN,僕らの最終戦争,UNKNOWN,A music video for the song Bokura no Saishuu S...,https://cdn.myanimelist.net/images/anime/1931/...


In [51]:
popular_df=df.merge(more_features,on="English name",how='left')

In [56]:
popular_df[popular_df["Scored By"] <250]

,English name,Score,Scored By,Other name,Genres,Synopsis,Image URL


In [57]:
#seems good

# now creating a quick ui , for it and showing top 50 animes